In [6]:

import os
import numpy as np
import torch as th
from stable_baselines3 import PPO
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.envs.unity_gym_env import UnityToGymWrapper
from gymnasium import Env spaces
from gymnasium import wrappers
from typing import Any List
from mlagents_envs.base_env import ActionTuple

SyntaxError: invalid syntax (2500068469.py, line 7)

In [ ]:
import os
import numpy as np
import torch as th
from stable_baselines3 import PPO
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.envs.unity_gym_env import UnityToGymWrapper
from gymnasium import Env spaces
from gymnasium import wrappers
from typing import Any List
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
class UnityToGymWrapper(Env):
    def __init__(self unity_env uint8_visual=True):
        super(UnityToGymWrapper self).__init__()
        self.unity_env = unity_env
        self.unity_env.reset()
        self.behavior_name = list(self.unity_env.behavior_specs.keys())[0]
        spec = self.unity_env.behavior_specs[self.behavior_name]
        
        # Definindo o espaço de ação e observação
        if spec.action_spec.continuous_size > 0:
            self.action_space = spaces.Box(low=-1 high=1 shape=(spec.action_spec.continuous_size) dtype=np.float32)
        else:
            raise ValueError(\O ambiente não possui ações contínuas.\)
        self.observation_space = spaces.Box(low=-np.inf high=np.inf shape=spec.observation_specs[0][0] dtype=np.float32)

        # Configuração para retornar imagens
        self.uint8_visual = uint8_visual
        self.render_mode = 'rgb_array'  # Definindo render_mode para rgb_array

    def reset(self seed=None options=None):
        self.unity_env.reset()
        decision_steps _ = self.unity_env.get_steps(self.behavior_name)
        return decision_steps.obs[0] {}  # Retornar uma tupla (observação info) como esperado pelo Gymnasium

    def step(self action):
        action = ActionTuple(np.array([action] dtype=np.float32))
        self.unity_env.set_actions(self.behavior_name action)
        self.unity_env.step()
        decision_steps terminal_steps = self.unity_env.get_steps(self.behavior_name)
        if len(decision_steps) > 0:
            obs = decision_steps.obs[0][0]
            reward = decision_steps.reward[0]
            done = False
        else:
            obs = terminal_steps.obs[0][0]
            reward = terminal_steps.reward[0]
            done = True
        return obs reward done {} {}

    def render(self mode='rgb_array'):
        decision_steps _ = self.unity_env.get_steps(self.behavior_name)
        return decision_steps.obs  # Retorna a imagem como um array RGB

    def close(self):
        self.unity_env.close()
    

# Caminho para o seu build do Unity
unity_env_path = '../../Starship2DEnv'

# Criando o ambiente Unity
unity_env = UnityEnvironment(file_name=unity_env_path no_graphics=False)

# Envolve o ambiente Unity com a interface Gymnasium
env = UnityToGymWrapper(unity_env uint8_visual=True)

# Set up video recording and monitoring
monitor_dir = \./exp/monitor\
os.makedirs(monitor_dir exist_ok=True)
# env = wrappers.RecordVideo(env video_folder=monitor_dir video_length=2000)

# Hiperparâmetros do PPO
n_steps = 2048
gamma = 0.99
learning_rate = 0.00025
ent_coef = 0.01
vf_coef = 0.5
max_grad_norm = 0.5
gae_lambda = 0.95
n_epochs = 100

# Inicializa o modelo PPO
model = PPO(\MlpPolicy\ env n_steps=n_steps gamma=gamma learning_rate=learning_rate
            tensorboard_log=\./runs/\
            ent_coef=ent_coef vf_coef=vf_coef max_grad_norm=max_grad_norm device='cpu')

# Treinamento do modelo
model.learn(total_timesteps=10000)

# Salva o modelo treinado no formato ONNX
model_path = 'ppo_unity_model_starship'
model.save(model_path)

# Convertendo o modelo para ONNX
def convert_to_onnx(model input_shape output_path):
    # Carrega o modelo novamente para garantir que seja compatível com PyTorch
    loaded_model = PPO.load(model_path env=env)

    loaded_model.policy.to('cpu')  # Coloca o modelo na CPU
    loaded_model.policy.eval()  # Coloca o modelo em modo de avaliação

    input_data = th.randn(*input_shape)
    torch.onnx.export(loaded_model.policy input_data output_path opset_version=12 export_params=True 
                        input_names=['input'] output_names=['output'] dynamic_axes={'input': {0: 'batch_size'} 'output': {0: 'batch_size'}})

onnx_model_path = 'ppo_unity_model_starship.onnx'
convert_to_onnx(model (1 *env.observation_space.shape) onnx_model_path)

print(f\Modelo salvo e convertido para ONNX em {onnx_model_path}\)
